In [ ]:
from pathlib import Path
import pandas as pd
import httpx
from typing import Optional
import asyncio

In [ ]:
GEOCODING_API_URL = 'https://maps.googleapis.com/maps/api/geocode/json'

In [ ]:
ROADS_URL = 'https://roads.googleapis.com/v1/nearestRoads'

In [ ]:
GEOCODING_API_KEY = 'FROM_ENV_VARIABLE'

In [ ]:
async def get_places(api_url: str, api_key: str, place_id: Optional[str]) -> Optional[list]:
    if place_id:
        response = httpx.get(url=api_url, params={
            'place_id': place_id,
            'key': api_key
        })
        return response.json().get('results')
    return None

In [ ]:
async def get_nearest_roads(url: str, api_key: str, points: str) -> Optional[list]:
    response = httpx.get(url=url, params={
        'points': points,
        'key': api_key
    })
    return response.json().get('snappedPoints')

In [ ]:
async def get_routes_for_places(place_id: str, places_url: str, key: str) -> Optional[list]:
    if places := await get_places(api_url=places_url, api_key=key, place_id=place_id):
        total_routes = []
        for place in places:
            if address_components := place['address_components']:
                if routes := [addr_comp for addr_comp in address_components if 'route' in addr_comp['types']]:
                    total_routes = total_routes + [route.get('short_name') for route in routes
                                                   if route.get('short_name')]
        return list(set(total_routes))
    return None

In [ ]:
async def get_routes_for_point(point: Optional[str], roads_url: str, places_url: str, key: str) -> Optional[str]:
    if point:
        if roads := await get_nearest_roads(url=roads_url, api_key=key, points=point):
            if roads_with_place_id := [road for road in roads if road.get('placeId')]:
                routes = await get_routes_for_places(place_id=roads_with_place_id[0].get('placeId'),
                                                                      places_url=places_url,
                                                                      key=key)
                return ','.join(list(set([route for route in routes if route])))
    return None

In [ ]:
COLUMNS_NAMES_MAPPING = {
    'מזהה': 'GUID',
    'תאריך דיווח': 'REPORT DATE',
    'סטטוס': 'STATUS',
    'סוג עבירה': 'VIOLATION TYPE',
    'מיקום': 'LOCATION',
    'סוג רכב': 'VEHICLE TYPE',
    'סוג לוחית רישוי': 'LICENSE PLATE TYPE',
    'טופס ערוך - עיר': 'CITY',
    'נ״צ ערוך': 'POINT'
}

In [ ]:
file_path = Path.cwd() / 'reports.xlsx'

In [ ]:
file_path

In [ ]:
raw_df = pd.read_excel(file_path)

In [ ]:
raw_df.head(5)

In [ ]:
raw_df.shape

In [ ]:
df = raw_df.copy(deep=True)

In [ ]:
df.rename(columns=COLUMNS_NAMES_MAPPING, inplace=True)

In [ ]:
df

In [ ]:
first_ten_df = df.iloc[:10]

In [ ]:
first_ten_df

In [ ]:
first_ten_df['ROUTE'] = await asyncio.gather(*[
    get_routes_for_point(point=point, roads_url=ROADS_URL, places_url=GEOCODING_API_URL, key=GEOCODING_API_KEY)
    for point in first_ten_df['POINT']
])

In [ ]:
first_ten_df[['LOCATION', 'ROUTE']]

In [ ]:
road_in_text = df[df['LOCATION'].str.contains('90')]

In [ ]:
first_ten_route_in_text = road_in_text.iloc[:10]

In [ ]:
first_ten_route_in_text

In [ ]:
first_ten_route_in_text